# Environment Setup
This file is intended to be run using a GPU with Google Colab. Go through each step and read the comments to learn how to set up parameters, preprocess/download the dataset, and train a model. 

In [ ]:
# ensure GPU is being used
!nvidia-smi

Tue Nov 18 22:17:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   46C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# mount Google Drive with nuScenes dataset .tgz file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# system information should be (approx):
# Python 3.12.11
# torch>=2.8.0
# torchvision>=0.23.0
# If there are issues, consider changing your runtime version to match the python version
!python3 --version
!pip freeze | grep torch

Python 3.12.12
torch==2.8.0+cu126
torchao==0.10.0
torchaudio==2.8.0+cu126
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.23.0+cu126


In [ ]:
# clone or pull the CenterFusionModern repository
import os
CF_ROOT = "/content/CenterFusionModern"
%cd /content/
repo_url = "https://github.com/mattbriggs04/CenterFusionModern.git"
init_submodules = False
if not os.path.exists(CF_ROOT):
  print(f"Cloning {CF_ROOT}")
  !git clone {repo_url}
  init_submodules = True
else:
  print(f"Updating {CF_ROOT}")
  %cd {CF_ROOT}
  !git pull -f origin main
  %cd ..

if init_submodules:
  print(f"Initializing submodules")
  %cd {CF_ROOT}
  !git submodule update --init --recursive
  %cd ..

/content
Cloning /content/CenterFusionModern
Cloning into 'CenterFusionModern'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 303 (delta 174), reused 215 (delta 93), pack-reused 0 (from 0)
Receiving objects: 100% (303/303), 117.11 KiB | 11.71 MiB/s, done.
Resolving deltas: 100% (174/174), done.
Initializing submodules
/content/CenterFusionModern
Submodule 'src/tools/nuscenes-devkit' (https://github.com/nutonomy/nuscenes-devkit) registered for path 'src/tools/nuscenes-devkit'
Cloning into '/content/CenterFusionModern/src/tools/nuscenes-devkit'...
Submodule path 'src/tools/nuscenes-devkit': checked out 'd9de17a73bdc06ce97a02f77ae7edb9b0406e851'
/content


In [ ]:
# pip install requirements
# Colab may ask you to restart the session, after restarting run all blocks above
# and this block again to ensure there is no errors
%cd CenterFusionModern
!pip install -r requirements.txt

/content/CenterFusionModern
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 135.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.0/316.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 54.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: shapely
    Found existing installation: shapely 2.1.2
    Uninstalling shapely-2.1.

# Data Preprocessing

In [ ]:
import os
dataset_path = "/content/drive/MyDrive/nuScenes/v1.0-mini.tgz"
dest_path = "/content/CenterFusionModern/data/"

# make a new directory if it does not already exist
# it should -- ensure CenterFusionModern is cloned
if not os.path.exists(dest_path):
  !mkdir -p f"{dest_path}"

# copy the compressed file
print("Copying dataset from Google Drive to Colab runtime (this will take a minute)...")
!cp "{dataset_path}" "{dest_path}"
print("Copy complete.")

Copying dataset from Google Drive to Colab runtime (this will take a minute)...
Copy complete.


In [ ]:
# unzip the file
print("Unzipping the dataset (this will take a minute)...")
!mkdir -p "{dest_path}/nuscenes/"
!tar -xzf "{dest_path}/v1.0-mini.tgz" -C "{dest_path}/nuscenes/"
print("Unzipping complete.")



Unzipping the dataset (this will take a minute)...
Unzipping complete.


In [ ]:
# clean up the copied .tgz file (optional)
!rm "{dest_path}/v1.0-mini.tgz"

# Camera-Radar Fusion Model

In [ ]:
# convert nuScenes dataset (6 radar sweeps)
!cd "{CF_ROOT}/src" && \
python3 convert_nuScenes.py

Using mini nuScenes dataset only
Either no command line arguments given or they're invalid.
Using NUM_SWEEPS = 6
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.381 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
scene_name scene-0103
scene_name scene-0916
reordering images
mini_val 486 images 4910 boxes
out_path ../data/nuscenes/annotations_6sweeps/mini_val.json
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.372 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
scene_name scene-0061
scene_name scene-0553
scene_name scene-0655
scene_name scene-0757


In [ ]:
# define useful parameters to be used between cells
# this is helpful if we are only testing a model, only training, only visualizing
# or any combination -- consistency is key

# number of epochs to train model
# if resuming, this should be the total number of epochs you want to train to
NUM_EPOCHS = 20

# model save paths
MODEL_SAVE_NAME = f"model_{NUM_EPOCHS}.pth" # this is how the script will name the model by default, even if resuming
MODEL_SAVE_PATH = f"{CF_ROOT}/exp/ddd/centerfusion/{MODEL_SAVE_NAME}"
DRIVE_CF_DIR_PATH = "/content/drive/MyDrive/CenterFusionModern"

# model load path, this is the path to the model you are evaluating
# will also resume from model load path
MODEL_LOAD_PATH = MODEL_SAVE_PATH
RESUME_TRAINING = True
if RESUME_TRAINING:
    MODEL_LOAD_PATH = f"{DRIVE_CF_DIR_PATH}/fusion_e20.pth"

# scale up the defaults to utilize the A100 at max potential
# learning rate should scale with batch size
BATCH_SIZE = 8 * 4
LEARNING_RATE = 2.5e-4 * 4



In [ ]:
# train fusion model
# export CUDA_DEVICE_ORDER=PCI_BUS_ID
!export CUDA_VISIBLE_DEVICES=0

# train for single gpu
if not RESUME_TRAINING:
    !cd /content/CenterFusionModern/src && \
    python3 main.py \
        ddd \
        --exp_id centerfusion \
        --arch dla_34 \
        --dla_node 'conv' \
        --shuffle_train \
        --train_split mini_train \
        --val_split mini_val \
        --val_intervals 5 \
        --nuscenes_att \
        --velocity \
        --batch_size {BATCH_SIZE} \
        --lr {LEARNING_RATE} \
        --num_epochs {NUM_EPOCHS} \
        --lr_step 50 \
        --save_point {NUM_EPOCHS // 2},{NUM_EPOCHS} \
        --gpus 0 \
        --not_rand_crop \
        --flip 0.5 \
        --shift 0.1 \
        --pointcloud \
        --radar_sweeps 6 \
        --pc_z_offset 0.0 \
        --pillar_dims 1.0,0.2,0.2 \
        --max_pc_dist 60.0 \
        --print_iter 0 \
        # --run_dataset_eval \ <-- evaluates dataset with metrics after every validation
        # --debug 4 \ <-- currently broken
        # --freeze_backbone \ <-- use when fine tuning heads


# resume training
else:
    !cd /content/CenterFusionModern/src && \
    python3 main.py \
        ddd \
        --exp_id centerfusion \
        --arch dla_34 \
        --dla_node 'conv' \
        --shuffle_train \
        --train_split mini_train \
        --val_split mini_val \
        --val_intervals 5 \
        --nuscenes_att \
        --velocity \
        --batch_size {BATCH_SIZE} \
        --lr {LEARNING_RATE} \
        --num_epochs {NUM_EPOCHS} \
        --lr_step 50 \
        --save_point {NUM_EPOCHS // 2},{NUM_EPOCHS} \
        --gpus 0 \
        --not_rand_crop \
        --flip 0.5 \
        --shift 0.1 \
        --pointcloud \
        --radar_sweeps 6 \
        --print_iter 0 \
        --pc_z_offset 0.0 \
        --pillar_dims 1.0,0.2,0.2 \
        --max_pc_dist 60.0 \
        --load_model {MODEL_LOAD_PATH} \
        --resume

DCN not currently available. Ensure --dla_node 'conv' is being used.
Fix size testing.
training chunk_sizes: [32]
input h w: 448 800
heads {'hm': 10, 'reg': 2, 'wh': 2, 'dep': 1, 'rot': 8, 'dim': 3, 'amodel_offset': 2, 'dep_sec': 1, 'rot_sec': 8, 'nuscenes_att': 8, 'velocity': 3}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'dep': 1, 'rot': 1, 'dim': 1, 'amodel_offset': 1, 'dep_sec': 1, 'rot_sec': 1, 'nuscenes_att': 1, 'velocity': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'dep': [256], 'rot': [256], 'dim': [256], 'amodel_offset': [256], 'dep_sec': [256, 256, 256], 'rot_sec': [256, 256, 256], 'nuscenes_att': [256, 256, 256], 'velocity': [256, 256, 256]}
Options = Namespace(task='ddd', dataset='nuscenes', test_dataset='nuscenes', exp_id='centerfusion', eval=False, debug=0, no_pause=False, demo='', load_model='/content/drive/MyDrive/CenterFusionModern/fusion_e10.pth', resume=True, gpus=[0], num_workers=4, not_cuda_benchmark=False, seed=317, not_set_cuda_env=False, print_iter=0, save

In [ ]:
# export a copy of fusion model to drive
# ensure path is correct, model name is "model_{numEpochs}"
!cp {MODEL_SAVE_PATH} {DRIVE_CF_DIR_PATH}
print(f"Copied {MODEL_SAVE_PATH} to {DRIVE_CF_DIR_PATH} successfully.")

Copied /content/CenterFusionModern/exp/ddd/centerfusion/model_20.pth to /content/drive/MyDrive/CenterFusionModern successfully.


In [ ]:
# test model metrics
# set the GPU to be visible
!export CUDA_VISIBLE_DEVICES=0

# Navigate to the source directory
# Run the evaluation script
!cd /content/CenterFusionModern/src && \
python3 test.py ddd \
    --exp_id centerfusion \
    --arch dla_34 \
    --dla_node 'conv' \
    --dataset nuscenes \
    --nuscenes_att \
    --velocity \
    --val_split mini_val \
    --run_dataset_eval \
    --pointcloud \
    --radar_sweeps 6 \
    --gpus 0 \
    --load_model {MODEL_LOAD_PATH}

DCN not currently available. Ensure --dla_node 'conv' is being used.
Fix size testing.
training chunk_sizes: [32]
input h w: 448 800
heads {'hm': 10, 'reg': 2, 'wh': 2, 'dep': 1, 'rot': 8, 'dim': 3, 'amodel_offset': 2, 'dep_sec': 1, 'rot_sec': 8, 'nuscenes_att': 8, 'velocity': 3}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'dep': 1, 'rot': 1, 'dim': 1, 'amodel_offset': 1, 'dep_sec': 1, 'rot_sec': 1, 'nuscenes_att': 1, 'velocity': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'dep': [256], 'rot': [256], 'dim': [256], 'amodel_offset': [256], 'dep_sec': [256, 256, 256], 'rot_sec': [256, 256, 256], 'nuscenes_att': [256, 256, 256], 'velocity': [256, 256, 256]}
Namespace(task='ddd', dataset='nuscenes', test_dataset='nuscenes', exp_id='centerfusion', eval=False, debug=0, no_pause=False, demo='', load_model='/content/drive/MyDrive/CenterFusionModern/fusion_e20.pth', resume=False, gpus=[0], num_workers=4, not_cuda_benchmark=False, seed=317, not_set_cuda_env=False, print_iter=0, save_all=Fals

In [ ]:
# run the visualizer to show the model output for a given image
!cd /content/CenterFusionModern/src && \
python3 visualizer.py ddd \
    --load_model {MODEL_LOAD_PATH} \
    --arch dla_34 \
    --dla_node 'conv' \
    --val_split mini_val \
    --pointcloud \
    --radar_sweeps 6 \
    --nuscenes_att \
    --velocity \
    --sample_id 20 \
    --save_path /content/sample_20_vis.jpg

DCN not currently available. Ensure --dla_node 'conv' is being used.
Fix size testing.
training chunk_sizes: [32]
Initializing visualizer...
input h w: 448 800
heads {'hm': 10, 'reg': 2, 'wh': 2, 'dep': 1, 'rot': 8, 'dim': 3, 'amodel_offset': 2, 'dep_sec': 1, 'rot_sec': 8, 'nuscenes_att': 8, 'velocity': 3}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'dep': 1, 'rot': 1, 'dim': 1, 'amodel_offset': 1, 'dep_sec': 1, 'rot_sec': 1, 'nuscenes_att': 1, 'velocity': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'dep': [256], 'rot': [256], 'dim': [256], 'amodel_offset': [256], 'dep_sec': [256, 256, 256], 'rot_sec': [256, 256, 256], 'nuscenes_att': [256, 256, 256], 'velocity': [256, 256, 256]}
Loading dataset split: mini_val
Dataset version 
==> initializing mini_val data from /content/CenterFusionModern/src/lib/../../data/nuscenes/annotations_6sweeps/mini_val.json, 
 images from /content/CenterFusionModern/src/lib/../../data/nuscenes ...
loading annotations into memory...
Done (t=1.11s)
creati

In [ ]:
# save the generated image to drive
!cp /content/sample_20_vis.jpg /content/drive/MyDrive/CenterFusionModern

#Camera Only Model
Use this model for generating a camera only baseline. Be careful to not overwrite a previous model -- the model name will be saved as `model_{num_epochs}.pth` regardless of if camera only or camera radar fusion is used.

It is recommended to save the model to drive and rename it to an identifiable name such as `camera_only_e20.pth`.

In [ ]:
# convert nuscenes dataset
!cd "{CF_ROOT}/src" && \
python3 convert_nuScenes.py --radar_sweeps 0

Using mini nuScenes dataset only
Using NUM_SWEEPS = 0
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.379 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
scene_name scene-0103
scene_name scene-0916
reordering images
mini_val 486 images 4910 boxes
out_path ../data/nuscenes/annotations/mini_val.json
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.363 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
scene_name scene-0061
scene_name scene-0553
scene_name scene-0655
scene_name scene-0757
scene_name scene-0796
scene_name scene-1077
scene_name scene-1094
s

In [ ]:
# define useful parameters to be used between cells
# this is helpful if we are only testing a model, only training, only visualizing
# or any combination -- consistency is key

# number of epochs to train model
# if resuming, this should be the total number of epochs you want to train to
NUM_EPOCHS = 20

# model save paths
EXPERIMENT_ID = "centerfusion_camera_only"
MODEL_SAVE_NAME = f"model_{NUM_EPOCHS}.pth" # DO NOT CHANGE --(changing this does not change the actual save name, it's hard-coded to match this)
MODEL_SAVE_PATH = f"{CF_ROOT}/exp/ddd/{EXPERIMENT_ID}/{MODEL_SAVE_NAME}"
DRIVE_CF_DIR_PATH = "/content/drive/MyDrive/CenterFusionModern"

# model load path, this is the path to the model you are evaluating
# will also resume from model load path
MODEL_LOAD_PATH = MODEL_SAVE_PATH
RESUME_TRAINING = True # use if testing a pretrained model as well
if RESUME_TRAINING:
    MODEL_LOAD_PATH = f"{DRIVE_CF_DIR_PATH}/camera_only_e20.pth"

# scale up the defaults to utilize the A100 at max potential
# learning rate should scale with batch size
BATCH_SIZE = 8 * 4
LEARNING_RATE = 2.5e-4 * 4

In [ ]:
# train camera-only model for comparison
!export CUDA_VISIBLE_DEVICES=0

# train for single gpu
if not RESUME_TRAINING:
    !cd /content/CenterFusionModern/src && \
    python3 main.py \
        ddd \
        --exp_id {EXPERIMENT_ID} \
        --arch dla_34 \
        --dla_node 'conv' \
        --shuffle_train \
        --train_split mini_train \
        --val_split mini_val \
        --val_intervals 5 \
        --nuscenes_att \
        --velocity \
        --batch_size {BATCH_SIZE} \
        --lr {LEARNING_RATE} \
        --num_epochs {NUM_EPOCHS} \
        --lr_step 50 \
        --save_point {NUM_EPOCHS // 2},{NUM_EPOCHS} \
        --gpus 0 \
        --not_rand_crop \
        --flip 0.5 \
        --shift 0.1 \
        --print_iter 0
else: # resuming training
    !cd /content/CenterFusionModern/src && \
    python3 main.py \
        ddd \
        --exp_id {EXPERIMENT_ID} \
        --arch dla_34 \
        --dla_node 'conv' \
        --shuffle_train \
        --train_split mini_train \
        --val_split mini_val \
        --val_intervals 5 \
        --nuscenes_att \
        --velocity \
        --batch_size {BATCH_SIZE} \
        --lr {LEARNING_RATE} \
        --num_epochs {NUM_EPOCHS} \
        --lr_step 50 \
        --save_point {NUM_EPOCHS // 2},{NUM_EPOCHS} \
        --gpus 0 \
        --not_rand_crop \
        --flip 0.5 \
        --shift 0.1 \
        --print_iter 0 \
        --resume

DCN not currently available. Ensure --dla_node 'conv' is being used.
Fix size testing.
training chunk_sizes: [32]
input h w: 448 800
heads {'hm': 10, 'reg': 2, 'wh': 2, 'dep': 1, 'rot': 8, 'dim': 3, 'amodel_offset': 2, 'nuscenes_att': 8, 'velocity': 3}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'dep': 1, 'rot': 1, 'dim': 1, 'amodel_offset': 1, 'nuscenes_att': 1, 'velocity': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'dep': [256], 'rot': [256], 'dim': [256], 'amodel_offset': [256], 'nuscenes_att': [256], 'velocity': [256]}
Options = Namespace(task='ddd', dataset='nuscenes', test_dataset='nuscenes', exp_id='centerfusion_camera_only', eval=False, debug=0, no_pause=False, demo='', load_model='', resume=False, gpus=[0], num_workers=4, not_cuda_benchmark=False, seed=317, not_set_cuda_env=False, print_iter=0, save_all=False, vis_thresh=0.3, debugger_theme='white', run_dataset_eval=False, save_imgs=[], save_img_suffix='', skip_first=-1, save_video=False, save_framerate=30, resize_video=

In [ ]:
# export a copy of fusion model to drive
# ensure path is correct, model name is "model_{num-epochs}"
!cp {MODEL_SAVE_PATH} {DRIVE_CF_DIR_PATH}
print(f"Copied {MODEL_SAVE_PATH} to {DRIVE_CF_DIR_PATH} successfully.")

Copied /content/CenterFusionModern/exp/ddd/centerfusion_camera_only/model_20.pth to /content/drive/MyDrive/CenterFusionModern successfully.


In [ ]:
# test model metrics -- camera only
# set the GPU to be visible
!export CUDA_VISIBLE_DEVICES=0

# Navigate to the source directory
# Run the evaluation script
!cd /content/CenterFusionModern/src && \
python3 test.py ddd \
    --exp_id centerfusion \
    --arch dla_34 \
    --dla_node 'conv' \
    --dataset nuscenes \
    --nuscenes_att \
    --velocity \
    --val_split mini_val \
    --run_dataset_eval \
    --gpus 0 \
    --load_model {MODEL_LOAD_PATH}

DCN not currently available. Ensure --dla_node 'conv' is being used.
Fix size testing.
training chunk_sizes: [32]
input h w: 448 800
heads {'hm': 10, 'reg': 2, 'wh': 2, 'dep': 1, 'rot': 8, 'dim': 3, 'amodel_offset': 2, 'nuscenes_att': 8, 'velocity': 3}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'dep': 1, 'rot': 1, 'dim': 1, 'amodel_offset': 1, 'nuscenes_att': 1, 'velocity': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'dep': [256], 'rot': [256], 'dim': [256], 'amodel_offset': [256], 'nuscenes_att': [256], 'velocity': [256]}
Namespace(task='ddd', dataset='nuscenes', test_dataset='nuscenes', exp_id='centerfusion', eval=False, debug=0, no_pause=False, demo='', load_model='/content/drive/MyDrive/CenterFusionModern/camera_only_e20.pth', resume=False, gpus=[0], num_workers=4, not_cuda_benchmark=False, seed=317, not_set_cuda_env=False, print_iter=0, save_all=False, vis_thresh=0.3, debugger_theme='white', run_dataset_eval=True, save_imgs=[], save_img_suffix='', skip_first=-1, save_video=F

In [ ]:
# run the visualizer to show the model output for a given image
!cd /content/CenterFusionModern/src && \
python3 visualizer.py ddd \
    --load_model {MODEL_LOAD_PATH} \
    --arch dla_34 \
    --dla_node 'conv' \
    --val_split mini_val \
    --nuscenes_att \
    --velocity \
    --sample_id 20 \
    --save_path /content/camera_only_id20.jpg

In [ ]:
# save the generated image to drive
!cp /content/camera_only_id20.jpg /content/drive/MyDrive/CenterFusionModern